In [3]:
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt

def load_point_cloud(filename):
    """ Load a PLY file and return as an open3d point cloud. """
    return o3d.io.read_point_cloud(filename)

def calculate_density(point_cloud, radius=0.1):
    """ Calculate the density of points within a specified radius. """
    pcd_tree = o3d.geometry.KDTreeFlann(point_cloud)
    densities = []
    for i in range(len(point_cloud.points)):
        [_, idx, _] = pcd_tree.search_radius_vector_3d(point_cloud.points[i], radius)
        densities.append(len(idx))
    return np.array(densities)

def detect_frontiers(densities, threshold):
    """ Detect frontier points based on a density threshold. """
    return np.where(densities < threshold)[0]

def visualize_point_cloud(point_cloud, frontier_indices):
    """ Visualize the point cloud with frontiers highlighted. """
    frontier_points = point_cloud.select_by_index(frontier_indices)
    non_frontier_points = point_cloud.select_by_index(frontier_indices, invert=True)

    # Set the colors
    frontier_points.paint_uniform_color([1, 0, 0])  # Red for frontiers
    non_frontier_points.paint_uniform_color([0, 0.5, 1])  # Blue for normal points

    # Combine back into a single cloud for visualization
    combined = frontier_points + non_frontier_points
    o3d.visualization.draw_geometries([combined])

# Example usage
filename = 'h_0_0.ply'
pcd = load_point_cloud(filename)
densities = calculate_density(pcd)
frontier_indices = detect_frontiers(densities, threshold=10)  # Adjust threshold as needed
visualize_point_cloud(pcd, frontier_indices)


In [8]:
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors import NearestNeighbors
import concurrent.futures

def load_and_subsample_point_cloud(filename, voxel_size=0.2):
    """ Load a PLY file, subsample it using a voxel grid filter, and return as an Open3D point cloud. """
    pcd = o3d.io.read_point_cloud(filename)
    return pcd.voxel_down_sample(voxel_size)

def calculate_density(point_cloud, radius=0.1):
    """ Calculate the density of points within a specified radius using KDTree from scikit-learn for batch processing. """
    points = np.asarray(point_cloud.points)
    nbrs = NearestNeighbors(radius=radius, algorithm='auto').fit(points)
    densities = nbrs.radius_neighbors_graph(points).sum(axis=1)
    return np.array(densities).flatten()

def detect_frontiers(densities, threshold):
    """ Detect frontier points based on a density threshold. """
    return np.where(densities < threshold)[0]

def visualize_point_cloud(point_cloud, frontier_indices):
    """ Visualize the point cloud with frontiers highlighted. """
    frontier_points = point_cloud.select_by_index(frontier_indices)
    non_frontier_points = point_cloud.select_by_index(frontier_indices, invert=True)

    # Set colors
    frontier_points.paint_uniform_color([1, 0, 0])  # Red for frontiers
    non_frontier_points.paint_uniform_color([0, 0.5, 1])  # Blue for normal points

    # Combine back into a single cloud for visualization
    combined = frontier_points + non_frontier_points
    o3d.visualization.draw_geometries([combined])

# Example usage
filename = 'h_merged_0.pcd'
pcd = load_and_subsample_point_cloud(filename)
densities = calculate_density(pcd)
frontier_indices = detect_frontiers(densities, threshold=10)  # Adjust threshold as needed
visualize_point_cloud(pcd, frontier_indices)

In [1]:
import open3d as o3d
import numpy as np

def load_point_cloud(filename):
    """ Load a PLY file and return as an open3d point cloud. """
    return o3d.io.read_point_cloud(filename)

def downsample_point_cloud(point_cloud, voxel_size=0.05):
    """ Downsample the point cloud using voxel downsampling. """
    return point_cloud.voxel_down_sample(voxel_size=voxel_size)

def calculate_density(point_cloud, radius=0.1):
    """ Calculate the density of points within a specified radius using batched KD-Tree queries. """
    pcd_tree = o3d.geometry.KDTreeFlann(point_cloud)
    points = np.asarray(point_cloud.points)
    densities = np.zeros(len(points), dtype=int)

    # Process in batches
    batch_size = 100  # Adjust batch size based on your system's memory capacity
    for i in range(0, len(points), batch_size):
        end = min(i + batch_size, len(points))
        for j in range(i, end):
            [_, idx, _] = pcd_tree.search_radius_vector_3d(point_cloud.points[j], radius)
            densities[j] = len(idx)
    return densities

def detect_frontiers(densities, threshold):
    """ Detect frontier points based on a density threshold. """
    return np.where(densities < threshold)[0]

def visualize_point_cloud(point_cloud, frontier_indices):
    """ Visualize the point cloud with frontiers highlighted in red and larger size. """
    # Create two point clouds: one for frontiers and one for non-frontiers
    frontier_points = point_cloud.select_by_index(frontier_indices)
    non_frontier_points = point_cloud.select_by_index(frontier_indices, invert=True)

    # Set colors and sizes
    frontier_points.paint_uniform_color([1, 0, 0])  # Red
    non_frontier_points.paint_uniform_color([0.5, 0.5, 0.5])  # Gray for non-frontiers
    frontier_points.points = o3d.utility.Vector3dVector(np.asarray(frontier_points.points) * 1.05)  # Enlarge frontier points

    # Combine back into a single cloud for visualization
    combined = frontier_points + non_frontier_points
    o3d.visualization.draw_geometries([combined])

# Example usage
filename = 'h_0_0.ply'
pcd = load_point_cloud(filename)
downsampled_pcd = downsample_point_cloud(pcd, voxel_size=0.02)  # Adjust voxel size as needed
densities = calculate_density(downsampled_pcd, radius=0.1)  # Adjust the radius as needed
frontier_indices = detect_frontiers(densities, threshold=10)  # Adjust threshold as needed
visualize_point_cloud(downsampled_pcd, frontier_indices)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [1]:
import open3d as o3d
import numpy as np
from concurrent.futures import ThreadPoolExecutor

def load_point_cloud(filename):
    """ Load a PLY file and return as an open3d point cloud. """
    return o3d.io.read_point_cloud(filename)

def downsample_point_cloud(point_cloud, voxel_size=0.05):
    """ Downsample the point cloud using voxel downsampling. """
    return point_cloud.voxel_down_sample(voxel_size=voxel_size)

def calculate_density_parallel(point_cloud, radius=0.1, num_workers=4):
    """ Calculate the density of points within a specified radius using parallel processing. """
    pcd_tree = o3d.geometry.KDTreeFlann(point_cloud)
    points = np.asarray(point_cloud.points)
    densities = np.zeros(len(points), dtype=int)

    def process_batch(batch):
        local_densities = []
        for point in batch:
            [_, idx, _] = pcd_tree.search_radius_vector_3d(point, radius)
            local_densities.append(len(idx))
        return local_densities

    # Create batches
    batch_size = len(points) // num_workers
    batches = [points[i:i + batch_size] for i in range(0, len(points), batch_size)]

    # Process in parallel
    with ThreadPoolExecutor(max_workers=num_workers) as executor:
        results = executor.map(process_batch, batches)
        for i, local_densities in enumerate(results):
            densities[i * batch_size:(i + 1) * batch_size] = local_densities

    return densities

def detect_frontiers(densities, threshold):
    """ Detect frontier points based on a density threshold. """
    return np.where(densities < threshold)[0]

def visualize_point_cloud(point_cloud, frontier_indices):
    """ Visualize the point cloud with frontiers highlighted in red and larger size. """
    # Create two point clouds: one for frontiers and one for non-frontiers
    frontier_points = point_cloud.select_by_index(frontier_indices)
    non_frontier_points = point_cloud.select_by_index(frontier_indices, invert=True)

    # Set colors and sizes
    frontier_points.paint_uniform_color([1, 0, 0])  # Red
    non_frontier_points.paint_uniform_color([0.5, 0.5, 0.5])  # Gray for non-frontiers
    frontier_points.points = o3d.utility.Vector3dVector(np.asarray(frontier_points.points) * 1.05)  # Enlarge frontier points

    # Combine back into a single cloud for visualization
    combined = frontier_points + non_frontier_points
    o3d.visualization.draw_geometries([combined])

# Example usage
filename = 'h_0_0.ply'
pcd = load_point_cloud(filename)
downsampled_pcd = downsample_point_cloud(pcd, voxel_size=0.02)  # Adjust voxel size as needed
densities = calculate_density_parallel(downsampled_pcd, radius=0.1)  # Adjust the radius as needed
frontier_indices = detect_frontiers(densities, threshold=10)  # Adjust threshold as needed
visualize_point_cloud(downsampled_pcd, frontier_indices)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [ ]:
    <node pkg="amcl" type="amcl" name="amcl">
    <rosparam file="$(find multi_explore)/params/amcl_params.yaml" command="load" />
        <param name="odom_frame_id" value="tb3_0/odom"/>
        <param name="base_frame_id" value="tb3_0/base_footprint"/>
        <remap from="static_map" to="/static_map"/>
        <param name="initial_pose_x" value="0.5"/>
        <param name="initial_pose_y" value="1.7"/>
        <param name="initial_pose_a" value="0.0"/>
    </node> 
